In [242]:
#!/Users/jessica/anaconda/lib/python3.5

import sklearn
import pandas as pd
import numpy as np

In [243]:
debating_data = pd.read_csv("data/debating_motions_aug9.csv")
header = debating_data.columns.values
debating_data = np.array(debating_data)
debating_data.shape

(1080, 22)

In [244]:
motion_and_infoslide = [debating_data[0,16], debating_data[0,17]]
' '.join(motion_and_infoslide)

'THW allow the trophy hunting of endangered animals Trophy hunting is the selective and regulated hunting of wild animals, for example lions. Typically, it takes place on reservations, particularly in Africa and North America.'

### Convert labels to binary vectors

In [245]:
Z = debating_data[:150,18:21]
Z[0]

array(['Environment', 'Economics', nan], dtype=object)

In [246]:
labels = [
'Art and Culture',
'Business',
'Criminal Justice System',
'Development',
'Economics','Education',
'Environment',
'Family',
'Feminism',
'Freedoms',
'Funny',
'International Relations',
'LGBT+',
'Media',
'Medical Ethics',
'Minority Communities'
'Morality'
'Politics'
'Religion'
'Science and Technology'
'Security, War and Military'
'Social Policy'
'Social Movements'
'Sports'
'Terrorism'
'The Human Experience'
]

len(labels)

16

In [247]:
Y_vector = np.zeros(shape=(150,16))
for i in range(150):
    for j in range(3):
        for k in range(16):
            if Z[i,j] == labels[k]:
                Y_vector[i,k] = 1

In [248]:
# Transform Ys into binary
Y_dict = dict.fromkeys(labels)
for i in range(16):
    Y_dict[labels[i]] = Y_vector[:,i]

In [249]:
Y_dict['Art and Culture']

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
        0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [250]:
# Choose category to train for
chosen_element = 'International Relations'
Y_for_element = Y_dict[chosen_element]

In [251]:
# Configure training set

X = debating_data[:100,16]
# Previous:
# Y = debating_data[:100,18]
Y = Y_for_element[:100]

In [252]:
# Test set

X_test = debating_data[100:150,16]
Y_test = Y_for_element[100:150]

In [253]:
# Check that data is in form we expect
X[99]

'TH regrets the decline of secular pan-Arab nationalism'

## Pipeline for Extracting Features and Training Classifier

In [254]:
# Import feature extraction (Fit-Transform) refinements
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [255]:
# Choose classifier
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()

In [256]:
from sklearn.pipeline import Pipeline
clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', classifier)
                    ])

In [257]:
clf = clf.fit(X, Y)

In [258]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X)
X_train_counts.shape

(100, 707)

In [259]:
count_vect.vocabulary_.get(u'house')

303

In [260]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(100, 707)

In [261]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(100, 707)

In [262]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, Y)

In [263]:
# Predict the outcome on a new document
docs_new = ['thw criminalise drugs', 'THW ban endangered animals',
           'ban drug taking in sports', 'THBT Saudi Arabia should nationalise its oil industry']
X_new_counts = CountVectorizer().transform(docs_new)
X_new_tfidf = TfidfTransformer().transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, category))

NotFittedError: CountVectorizer - Vocabulary wasn't fitted.

In [ ]:
## Evaluation of the performance on the test set
import numpy as np

predicted = clf.predict(X_test)
np.mean(predicted == Y_test)